In [214]:
filename = "./dataset/forCLFnohash8000.csv"
with open(filename) as f:
    content = f.readlines()

##Remove Labels
del(content[0])

##Remove '\n' elements
content = [x for x in content if x != '\n']

##Striping Whitespaces
content = [x.strip() for x in content]

##Spilting by ','
content = [x.split(',') for x in content]
##print(content)

In [215]:
docs = [x[0] for x in content]
##docs = [x[0] for x in content[0:125]] + [x[0] for x in content[125:250]]
##print(docs)

In [216]:
from pythainlp.corpus import stopwords
from pythainlp.tokenize import word_tokenize
import re

def tokenizer(sentence):
    pattern = re.compile(r"[^\u0E00-\u0E4C]|^'|'$|''|\u0E46|\u0E3F")
    char_to_remove = re.findall(pattern, sentence)
    list_with_char_removed = [char for char in sentence if not char in char_to_remove]
    result_string = ''.join(list_with_char_removed)
    return word_tokenize(result_string, engine='deepcut')

In [217]:
tokenized_docs = [tokenizer(x) for x in docs]
##print(tokenized_docs)

In [218]:
word_set = [set(x) for x in tokenized_docs]
##print(word_set)

In [219]:
w = word_set[0].union(word_set[1])
for x in range(len(word_set)):
    w = w.union(word_set[x])
##'โมโห' in w

In [220]:
wordDictList = []
for doc in tokenized_docs:
    wordDict = dict.fromkeys(w,0)
    for word in doc:
        wordDict[word]+=1
    wordDictList.append(wordDict)
##wordDictList
##'โมโห' in wordDictList[0]

In [221]:
import pandas as pd
pd.DataFrame(wordDictList)

,,ก,กก,กกก,กกกก,กกกกก,กกกกกก,กกกกกกก,กกกกกกกก,กกกกกกกกก,...,ไอ้เชี้ยเหี้ยยย,ไอ้เม็ด,ไอ้เหี้ย,ไอ้เหี้ยยยยยยยยย,ไอ้แซง,ไอ้โป่ง,ไฮคิว,ไฮดร้าพอ,ไเพราะ,ไุ่้ะ
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

In [223]:
tfDict_list = []
for word,doc in list(zip(wordDictList, tokenized_docs)):
    tfDict_list.append(computeTF(word,doc))
##tfDict_list

In [224]:
def computeIdf(docList):
    import math
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(),0)
    for doc in docList:
        for word, val in doc.items():
            if val>0:
                idfDict[word]+=1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log(N/float(val))
        
    return idfDict

In [225]:
idfs = computeIdf(wordDictList)

In [226]:
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word,val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [227]:
tfidf_list = []
for tfDict in tfDict_list:
    tfidf_list.append(computeTFIDF(tfDict,idfs))
##tfidf_list

In [228]:
import pandas as pd
pd.DataFrame(tfidf_list)

,,ก,กก,กกก,กกกก,กกกกก,กกกกกก,กกกกกกก,กกกกกกกก,กกกกกกกกก,...,ไอ้เชี้ยเหี้ยยย,ไอ้เม็ด,ไอ้เหี้ย,ไอ้เหี้ยยยยยยยยย,ไอ้แซง,ไอ้โป่ง,ไฮคิว,ไฮดร้าพอ,ไเพราะ,ไุ่้ะ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [275]:
import numpy as np
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import recall_score, precision_score, f1_score
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler

X = []
for l in tfidf_list:
    temp = []
    for key, val in l.items():
        temp.append(val)
    X.append(temp)

X = np.array(X)
##X = normalize(X)

scaler = MinMaxScaler()
X = scaler.fit(X).transform(X)

y = [ 0 for i in range(len(tokenized_docs))]
for i in range(int(len(tokenized_docs)/8)):
    y[i]=1
y = np.array(y)

##print(X)

skf = StratifiedKFold(n_splits=5, random_state=30, shuffle=True)
##skf.get_n_splits(X, y)
for train_index, test_index in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    tclf = svm.LinearSVC(class_weight='balanced').fit(X_train,y_train)
    predict = tclf.predict(X_test)
    
    print(predict)
    print(y_test)
    print(recall_score(y_test,predict),precision_score(y_test,predict),f1_score(y_test,predict))

[0 0 0 ... 0 0 0]
[1 1 1 ... 0 0 0]
0.215 0.25748502994011974 0.23433242506811985
[1 1 0 ... 0 0 0]
[1 1 1 ... 0 0 0]
0.245 0.32450331125827814 0.2792022792022792
[0 0 0 ... 0 0 0]
[1 1 1 ... 0 0 0]
0.21 0.25 0.2282608695652174
[0 0 0 ... 0 0 0]
[1 1 1 ... 0 0 0]
0.255 0.32075471698113206 0.2841225626740947
[1 1 0 ... 0 0 0]
[1 1 1 ... 0 0 0]
0.2 0.272108843537415 0.23054755043227665


In [230]:
## Create Emotion Dictionary
angry_dict = [
    'โทสะ','วิโรธ','ความโกรธ','ความฉุนเฉียว','ความขัดเคือง','ขัดแค้น','ขึ้งเคียด','ขึ้งโกรธ',
    'ดาลเดือด','ทำให้โกรธ','ทำให้ฉุนเฉียว','ทำให้เกิดโทสะ','ยั่วโมโห','โกรธ','ฉุนเฉียว','เคือง',
    'แค้น','เดือด','โมโห']
anticipate_dict = [
    'คาด','ตั้งตาคอย','หยั่งทราบ','คาดหมาย','มุ่งหวัง','ชิงทำ','ทำล่วงหน้า','คาดคิด','คาดหวัง',
    'คาดเดา','ตั้งตารอ','ประมาณการณ์','มาดหมาย','เป็นที่คาดหวัง']

In [231]:
def findDictFeature(dictionary):
    N = len(wordDictList)
    dict_feature = [0 for i in range(N)]
    for idx, wordDict in enumerate(wordDictList):
        for word, val in wordDict.items():
            if val > 0:
                if word in dictionary:
                    dict_feature[idx] = 1
                    print(idx,word)
    return np.array(dict_feature).reshape(N,1)

In [232]:
angry_dict_feature = findDictFeature(angry_dict)

28 โกรธ
153 โกรธ
204 โมโห
229 โมโห
287 โกรธ
300 โมโห
318 โมโห
386 โกรธ
421 เดือด
480 โกรธ
484 โกรธ
582 โกรธ
598 โมโห
610 โมโห
634 โกรธ
675 โกรธ
841 โกรธ
978 โมโห
1240 เคือง
2062 โกรธ
2445 โกรธ
2530 โมโห
2741 โกรธ
2937 โมโห
3326 โกรธ
3447 โกรธ
3670 โกรธ
3809 โกรธ
3866 โมโห
4177 โกรธ
5673 โกรธ
5673 โมโห
6109 โกรธ
6676 โมโห
6827 โกรธ
7261 โกรธ


In [233]:
anticipate_dict_feature = findDictFeature(anticipate_dict)

1067 คาดหวัง
1086 คาด
1182 คาดหวัง
1261 คาด
1542 คาดหวัง
1891 คาดหวัง
2263 คาดหวัง
3054 คาดคิด
3605 คาดหวัง
4496 คาดหวัง
5127 คาด
5167 คาด
5356 คาดหวัง
5602 คาดหวัง
6232 คาด
6940 คาดหมาย
7004 คาด
7601 คาด


In [276]:
Xnew = np.concatenate((X,angry_dict_feature,anticipate_dict_feature), axis=1)

In [277]:
##Xnew = normalize(Xnew)
##scaler = MinMaxScaler()
##Xnew = scaler.fit(Xnew).transform(Xnew)

adf = normalize(angry_dict_feature)
scaler = MinMaxScaler()
adf = scaler.fit(adf).transform(adf)

skf = StratifiedKFold(n_splits=5, random_state=30, shuffle=True)
##skf.get_n_splits(X, y)
for train_index, test_index in skf.split(adf, y):
    #print("TRAIN:", X[train_index], "TEST:", X[test_index])
    X_train, X_test = adf[train_index], adf[test_index]
    y_train, y_test = y[train_index], y[test_index]
    tclf = svm.LinearSVC(class_weight='balanced').fit(X_train,y_train)
    predict = tclf.predict(X_test)
    
    print(predict)
    print(y_test)
    print(recall_score(y_test,predict),precision_score(y_test,predict),f1_score(y_test,predict))

[0 0 0 ... 0 0 0]
[1 1 1 ... 0 0 0]
0.015 0.5 0.029126213592233007
[0 0 0 ... 0 0 0]
[1 1 1 ... 0 0 0]
0.01 0.5 0.0196078431372549
[0 0 0 ... 0 0 0]
[1 1 1 ... 0 0 0]
0.025 0.625 0.04807692307692307
[0 0 0 ... 0 0 0]
[1 1 1 ... 0 0 0]
0.02 0.4 0.03809523809523809
[0 0 0 ... 0 0 0]
[1 1 1 ... 0 0 0]
0.02 0.5714285714285714 0.03864734299516908


In [236]:
angry_dict_feature.shape

(8000, 1)

In [237]:
test = np.zeros(250).reshape(250,1)
for i in range (250):
    if i<125: 
        test[i]=1
    else: 
        test[i]=0
test.shape

(250, 1)

In [238]:
Xnew2 = np.concatenate((Xnew,test), axis=1)

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [278]:
scaler = MinMaxScaler()
Xnew = scaler.fit(Xnew).transform(Xnew)
skf2 = StratifiedKFold(n_splits=5, random_state=30, shuffle=True)

for train_index, test_index in skf.split(Xnew, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train2, X_test2 = Xnew[train_index], Xnew[test_index]
    y_train2, y_test2 = y[train_index], y[test_index]
    tclf = svm.LinearSVC(class_weight='balanced').fit(X_train2,y_train2)
    predict2 = tclf.predict(X_test2)
    
    print(predict2)
    print(y_test2)
    print(recall_score(y_test2,predict2),precision_score(y_test2,predict2),f1_score(y_test2,predict2))

[0 0 0 ... 0 0 0]
[1 1 1 ... 0 0 0]
0.215 0.2606060606060606 0.23561643835616441
[1 1 0 ... 0 0 0]
[1 1 1 ... 0 0 0]
0.245 0.3202614379084967 0.27762039660056653
[0 0 0 ... 0 0 0]
[1 1 1 ... 0 0 0]
0.225 0.2631578947368421 0.2425876010781671
[0 0 0 ... 0 0 0]
[1 1 1 ... 0 0 0]
0.255 0.32075471698113206 0.2841225626740947
[1 1 0 ... 0 0 0]
[1 1 1 ... 0 0 0]
0.205 0.2751677852348993 0.23495702005730656


In [240]:
def train_model(X,y,n):
    features = X
    label =y
    
    R = 0.0
    P = 0.0
    F = 0.0
    
    skf = StratifiedKFold(n_splits=n, random_state=30, shuffle=True)
    
    for train_index, test_index in skf.split(features, label):
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = label[train_index], label[test_index]
        tclf = svm.LinearSVC(class_weight='balanced').fit(X_train,y_train)
        predict = tclf.predict(X_test)
        
        r=recall_score(y_test,predict)
        p=precision_score(y_test,predict)
        f=f1_score(y_test,predict)
        
        R = R + r
        P = P + p
        F = F + f
        
        print('Prediction')
        print(predict)
        print('y')
        print(y_test)
        print('scores: recall, precision, f1')
        print(r,p,f)
        print()
        
    print()
    print('Average')
    print(R/n,P/n,F/n)

In [279]:
train_model(X,y,5)

Prediction
[0 0 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.215 0.25748502994011974 0.23433242506811985

Prediction
[1 1 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.245 0.32450331125827814 0.2792022792022792

Prediction
[0 0 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.21 0.25 0.2282608695652174

Prediction
[0 0 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.255 0.32075471698113206 0.2841225626740947

Prediction
[1 1 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.2 0.272108843537415 0.23054755043227665


Average
0.225 0.284970380343389 0.2512931373883976


In [242]:
train_model(Xnew,y,5)

Prediction
[0 0 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.215 0.2606060606060606 0.23561643835616441

Prediction
[1 1 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.245 0.3202614379084967 0.27762039660056653

Prediction
[0 0 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.225 0.2631578947368421 0.2425876010781671

Prediction
[0 0 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.255 0.32075471698113206 0.2841225626740947

Prediction
[1 1 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.205 0.2751677852348993 0.23495702005730656


Average
0.229 0.28798957909348616 0.2549808037532598


In [243]:
## Create dictionary from NRC Emotion Lexicon
fname = './dataset/NRC-Emotion-Lexicon.xlsx'
df = pd.read_excel(fname)

emowords = np.array(df['Thai'])
emowords

array(['ผงะ', 'ลูกคิด', 'ละทิ้ง', ..., 'เกี่ยวกับสัตววิทยา', 'สัตววิทยา',
       'ซูม'], dtype=object)

In [244]:
emowords.shape

(14182,)

In [245]:
emodict = findDictFeature(emowords)

0 สอน
0 ดู
0 สัก
0 ก่อน
0 คน
1 รับ
2 ต่อ
2 ทำ
2 เป็น
2 แกล้ง
2 คน
3 อยู่
3 บอก
3 พระ
3 ตอน
3 กรุ
4 แถว
4 แสดง
4 ซื้อ
4 มา
4 ข้าว
4 อาหาร
4 ใช่
4 กิน
4 คน
4 ออก
4 ลง
4 ห้อง
5 หก
5 ด่า
6 กระโดด
6 อย่าง
6 หน้า
6 เป็น
6 กรรมการ
6 ตอแหล
7 หมู
7 แมว
7 หนวด
7 เบี้ยว
7 กัด
7 โทรศัพท์
7 ทำไม
7 หมา
7 หลอด
8 ช่าง
9 พูด
9 กระทำ
9 กับ
9 แตกต่าง
9 คำ
10 แรก
10 ให้
10 ปาก
10 ดัง
10 ทำ
10 บอก
10 แบบ
10 ตาย
10 แฟน
10 เรียก
10 ชื่อ
10 เพื่อน
10 ตีน
10 อารมณ์
11 ฟัน
11 หา
12 เปิด
12 ไฟ
12 อยู่
12 ตอน
12 มา
12 สาม
12 ดับ
12 โทรศัพท์
12 อ่าน
13 ให้
13 ใหม่
13 เป็น
13 ตอน
13 มา
13 หลุด
13 ค้าง
14 คู่
14 เหยียบ
14 ใส่
14 ซื้อ
14 มา
15 ให้
15 ชอบ
15 พ่อ
15 ดี
15 เป็น
15 ชื่อ
15 ตรง
15 คน
15 แม่
16 ชนะ
16 เป่า
16 ทำไม
17 ให้
17 ทำ
17 เป็น
17 ลูกศร
17 มี
17 ปะ
17 วาด
18 เป็น
19 หัว
19 ดี
19 ทำ
19 งาน
19 หี
19 แก่น
20 ยืม
20 ผม
20 ยาว
21 ร้อน
22 ดู
22 บอก
22 มา
22 เปรี้ยว
22 มะม่วง
23 ขับ
23 รถ
23 ไอ
23 บอก
23 โทร
23 มา
23 จอด
23 ตี
23 ก่อน
23 หม้อ
23 เพื่อน
23 ทิ้ง
23 เสร็จ
23 ลง
24 หื่น
24 กับ
24 คน
26 เรื่อง


282 ทิ้ง
282 ยาม
282 สวย
283 สอบ
283 เสร็จ
284 ให้
284 เรื่อง
284 จัด
284 ชีวิต
284 ใช่
284 วัน
284 มี
285 รับ
285 เป็น
285 งาน
285 ตอน
285 มา
285 ส่ง
285 โทรศัพท์
285 ใช้
285 เร่ง
286 ก่อน
286 สิว
286 ออก
287 แรก
287 โกรธ
287 ทำ
287 ล้าน
287 ตอน
287 มา
287 เล่น
288 อารมณ์
289 ดี
289 มือ
289 ถือ
289 รัก
290 รูป
290 ใหม่
290 ป่า
290 ทำ
290 แบบ
290 แฟน
290 สถานะ
290 ชื่อ
290 คน
291 เข้าใจ
293 ใช่
294 ชอบ
294 เขา
294 เป็น
294 เหนื่อย
294 เอาเปรียบ
294 คน
294 คิด
296 มาก
296 ดี
296 แบบ
296 ทัน
296 ตาย
296 กรรม
296 ตาม
296 สังคม
296 คน
297 เก็บ
297 ให้
297 เดียว
297 แบบ
297 มา
297 คน
297 เล่น
298 ติด
299 พูด
299 ให้
299 เดือน
299 บอก
299 รัก
299 เห็น
299 ตลอด
299 อากาศ
300 อยู่
300 ดี
300 ด้าน
300 ตาย
300 ใช้
301 พูด
301 ให้
301 อย่าง
301 ทำ
301 กอง
301 งาน
301 ขั้น
301 ใช้
301 วิเคราะห์
301 ง่าย
302 ตอบ
303 ให้
303 บอก
303 เป็น
303 กลับ
303 มา
303 คน
304 โกหก
305 เกลียด
305 มาก
305 ขนาด
305 คน
306 ทำไม
307 เก่ง
307 มาก
308 เรียน
308 วัน
309 บ้าน
309 ออก
310 พูด
310 ไม่ลืมหูลืมตา
310 ถูก
31

546 เต็ม
546 ปิด
546 นอน
546 ปวด
546 เล่น
547 หน้า
547 สอบ
547 เห็น
547 คน
548 บ้า
548 ดอก
548 เด็ก
549 อย่าง
549 เรียน
549 ดี
549 อดทน
549 ชีวิต
549 มี
549 ทิ้ง
549 ลง
550 ขอ
550 สัก
550 รวย
550 คน
552 รถ
552 หน้า
552 ถือ
552 หัวเราะ
552 เครื่อง
552 กระชาก
552 สับ
552 ว่าง
552 เล่น
553 อยู่
553 หน้า
553 ยุง
553 ตอน
553 มา
553 สถานะ
553 ตี
553 ตรง
554 มาก
554 ดอก
554 กลับ
554 มา
554 กับ
555 รับผิดชอบ
555 อยู่
555 ทำ
555 เป็น
555 ตอน
555 นิสัย
555 คน
555 คิด
556 หิว
557 ศิลปิน
557 เป็น
557 เสียใจ
557 รัก
557 แฟน
557 ด่า
557 เพื่อน
557 คน
558 ใกล้
558 อยู่
558 นั่ง
558 อิจฉา
558 แบบ
558 ลำบาก
558 เห็น
558 บาย
559 กับ
560 สอบ
560 เสร็จ
562 ลืม
562 เหลือ
562 เหยียบ
563 ให้
563 ทำ
563 งาน
563 ราคา
563 ซื้อ
563 มา
563 เด้ง
563 เครื่อง
563 ออก
564 ดู
564 บอก
564 สนใจ
564 กับ
565 คน
566 ดู
566 มุม
566 เป็น
566 หนี
566 เอ็น
566 คอ
566 สภาพ
567 กับ
567 วัน
568 ยาก
569 ให้
570 อย่าง
571 อย่าง
571 มาก
571 ขว้าง
571 หยิบ
571 ห้อง
572 ทำไม
573 ยาก
573 ดี
573 เป็น
573 คน
573 เลว
574 วัน
574 คน
575 แร

802 ปี
803 พอ
803 ประเภท
803 แฟน
803 ด่า
803 ลง
803 บ่น
803 ทีม
803 เจ็บ
804 อยู่
804 บอก
804 มี
804 คน
805 เรียน
805 เกรด
805 ตอน
805 หลอก
805 ลง
806 คู่
806 อยู่
806 ปัญญา
806 เดียว
806 ใส่
806 ซื้อ
806 รองเท้า
806 ใส
806 มี
806 ตีน
807 ชอบ
807 หัว
807 ล้าน
807 ขนาด
808 หัว
808 ดี
808 เป็น
808 ตอน
808 เห็น
808 แน่นอน
809 ขวาง
809 อย่าง
809 ร้อน
809 ขัด
809 แฟน
810 ให้
810 อย่าง
810 ลุก
810 ทำ
810 เดียว
810 น้ำ
810 นอน
811 ขัดแย้ง
811 หยุด
811 ปิด
811 กับ
811 เศรษฐกิจ
811 ประเทศ
811 สังคม
811 เสมือน
812 แปลก
812 เขา
812 หยุด
812 ยอม
812 แฟน
812 กับ
812 มี
812 คน
813 ส้น
813 ตีน
814 ให้
814 ดับ
815 ให้
815 เรื่อง
815 นั่ง
815 ใส่
815 มา
815 ปัญหา
815 แท้
815 คน
816 รูป
816 มา
817 ไหม
817 ตบ
817 เห็น
818 ตัดสิน
818 ก่อ
818 ชีวิต
818 ร้าย
818 งง
818 คน
819 ดู
819 รูป
819 ให้
819 มา
819 ลง
820 มา
821 เรื่อง
821 มี
821 ด่า
822 ให้
822 ฝน
822 ตอน
822 ตก
822 หนัก
823 เปิด
823 ติด
824 หญิง
824 มารยา
824 มา
824 แพ้
825 ขับ
825 ให้
825 มาก
825 ทำ
825 สนใจ
825 งาน
825 มา
825 บริการ
825 มี
825 กร

1035 ฟัง
1035 ทำ
1035 รัก
1036 รับ
1037 มี
1038 อดทน
1039 ดู
1039 ละคร
1039 คิด
1040 ต่อ
1040 ดิน
1040 มา
1040 วัน
1041 ชอบ
1042 ใส่
1042 แบบ
1045 หวัง
1045 ขาย
1045 ให้
1045 บัตร
1045 อยู่
1045 ราคา
1045 หนู
1045 มี
1045 คน
1046 รุ่น
1046 กล้า
1046 โยน
1047 เด็ด
1047 บ้า
1047 เขา
1047 ขอ
1047 เงิน
1047 ซื้อ
1047 เผื่อ
1047 กับ
1047 ก่อน
1047 หวย
1047 หา
1048 ครึ่ง
1048 ชม
1049 ให้
1049 แชมป์
1049 บาง
1049 คว้า
1050 แรก
1050 เสื้อ
1050 ทำ
1050 ตอน
1050 มา
1050 ขาว
1050 ใช่
1050 คล้าย
1050 คิด
1051 ปล่อย
1051 มา
1052 ไหล
1052 น้ำ
1052 มา
1052 สุข
1052 ตา
1053 ให้
1053 ขอ
1053 วัน
1053 มี
1054 บัตร
1054 แพง
1054 เสื้อ
1054 ขอ
1054 ถูก
1054 ยอม
1054 มา
1054 ชาย
1055 ให้
1056 หวัง
1056 ให้
1056 รับ
1056 ร่วม
1056 ดี
1056 เป็น
1056 วัน
1056 มี
1056 สิ่ง
1056 ภูมิใจ
1057 หวัง
1057 ให้
1057 ค่า
1057 สูง
1057 ลม
1057 ตาย
1057 มี
1057 อัน
1057 สิ่ง
1057 ฆ่า
1057 กระชาก
1057 แย่งชิง
1057 ลง
1058 มา
1059 หวัง
1059 ให้
1059 เขา
1059 เพลง
1059 อยู่
1059 ฟัง
1059 กลับ
1059 มา
1059 สุข
1059 ฝัน
1059 

...

6584 คำ
6584 สาว
6585 เดิน
6585 ขวา
6585 เป้า
6585 ซ้าย
6586 หน้า
6586 กก
6586 ทำ
6586 เป็น
6586 เต็ม
6586 ศึกษา
6586 งาน
6586 สั้น
6586 มา
6586 ผม
6586 รายงาน
6586 วัน
6586 มี
6586 กระโปรง
6586 แต่ง
6587 หัว
6588 บอก
6588 ใช่
6588 อ่าน
6588 คน
6589 ขับ
6589 ใกล้
6589 รถ
6589 ฝน
6589 บ้าน
6589 มาก
6589 พอ
6589 ถนน
6589 กลับ
6589 แวว
6589 ขนาด
6589 แห้ง
6589 ตก
6589 ทาง
6589 มี
6589 ตลอด
6589 สอง
6590 ช่อง
6590 ไอ้
6591 ตอน
6591 ผม
6592 ชำรุด
6592 สูง
6592 บัตร
6592 ทำ
6592 ตอน
6593 พูด
6593 สังเกต
6593 ศัพท์
6593 แพทย์
6594 เปิด
6594 หิว
6594 มา
6594 ข้าว
6598 ลูกโป่ง
6599 นางสาว
6599 เด็ก
6599 เป็น
6599 สมัย
6599 แบบ
6599 เริ่ม
6599 ก่อน
6599 ใหญ่
6599 รู้สึก
6599 มี
6599 คิด
6600 ชื่อ
6601 เปลี่ยน
6601 มี
6602 บังเอิญ
6602 มา
6603 เลิก
6603 นำ
6603 ห่าน
6604 แบบ
6605 หน้า
6606 เก็บ
6606 เที่ยง
6606 ขาด
6606 ต่อ
6606 สลัด
6606 พอ
6606 ยุง
6606 บิน
6606 ตอน
6606 ตาย
6606 ทำไม
6606 ตบ
6606 เลือด
6606 ออก
6607 เสียง
6607 มาก
6607 รัก
6607 มา
6607 สวย
6608 ให้
6608 อ่อน
6608 ใบ
6608 กิน
6

6939 คน
6940 เหนือ
6940 เรื่อง
6940 มาก
6940 คน
6940 เล่น
6940 แต่ง
6941 เป็ด
6941 มาก
6941 อายุ
6941 ลำดับ
6941 ตาม
6942 เพศ
6942 เดียว
6942 แบบ
6944 สวย
6945 ดู
6945 โฆษณา
6945 จริง
6945 ทัน
6945 ช่อง
6945 กวน
6945 ตัด
6946 หนู
6946 เพื่อน
6947 เขา
6947 เป็น
6947 สื่อ
6947 ไหม
6947 ใช่
6948 มี
6948 เพื่อน
6948 คิ้ว
6948 ลง
6949 มี
6949 นาที
6949 นับ
6950 ข่าว
6950 นิ่ง
6950 ท่าน
6950 ทำไม
6950 สิบ
6951 เต็ม
6951 มา
6951 เหงือก
6952 เป็ด
6952 อยู่
6952 หนังสือ
6952 ซาก
6952 ทำ
6952 ย่าง
6952 ตอน
6952 จุด
6952 อ่าน
6956 มาก
6956 ตอน
6956 เส้น
6956 สาม
6956 วิ่ง
6957 ทำไม
6957 ตี
6957 วัน
6958 ถนน
6958 ทำไม
6958 วัน
6958 ปกติ
6959 เดิน
6959 ขอ
6959 นั่ง
6959 มา
6959 มี
6959 ลง
6959 สาว
6960 มี
6961 ทำไม
6962 ร้องไห้
6962 แบบ
6962 โทร
6962 มา
6962 สะอื้น
6964 ขาย
6964 ลุก
6964 เป็น
6964 น้ำ
6964 ข้าว
6964 ใช่
6964 ทาง
6964 กิน
6965 หนังสือ
6965 ทำ
6965 มา
6965 วาง
6965 จับ
6965 ชม
6967 คะแนน
6967 ทำ
6968 อยู่
6968 วัน
6969 บ้า
6969 อยู่
6969 เป็น
6969 อสูร
6969 มี
6971 มา
6971 แต่งงาน
69

7194 มี
7194 สิ่ง
7195 บังคับ
7195 ให้
7195 ชีวิต
7195 คน
7196 ดู
7196 รูป
7196 ชอบ
7196 เขา
7196 จิตใจ
7196 จริง
7196 คน
7197 มี
7197 ทน
7197 คน
7198 เงา
7199 ดู
7199 ให้
7199 รับ
7199 มาก
7199 ดี
7199 ทำ
7199 คิด
7200 กับ
7201 กับ
7201 ผม
7202 เชื่อ
7203 เต็ม
7204 ให้
7204 รับ
7204 เดียว
7204 วัน
7204 รู้สึก
7204 มี
7204 คน
7204 แน่นอน
7205 โลก
7205 ร้าย
7205 ชื่อ
7205 อาวุธ
7207 เก็บ
7207 ลืม
7207 ให้
7207 อยู่
7207 ทำ
7207 เวลา
7207 รัก
7207 กลับ
7207 ทรมาน
7207 ตาย
7207 อาจ
7207 โง่
7207 วัน
7207 จำ
7207 ปวด
7207 ตลอด
7207 ทน
7207 ตรง
7207 คน
7207 เจ็บ
7209 ตอน
7209 มา
7209 เริ่ม
7209 ชัดเจน
7209 รู้สึก
7210 พูด
7210 อยู่
7210 เป็น
7210 แฟน
7210 เชื่อ
7210 คน
7211 ห่าง
7211 อยู่
7211 เวลา
7211 รัก
7211 เหิน
7211 มา
7211 ทาง
7211 วัน
7211 คำ
7212 เป็น
7212 เมล็ด
7212 ประโยค
7212 ใหญ่
7212 สอง
7212 คำ
7213 เพลง
7213 เต็ม
7213 กลับ
7213 น้ำ
7213 มา
7213 สาม
7213 ตา
7214 ข่าว
7214 บุคคล
7214 เกา
7214 โลก
7214 เพลง
7214 แบบ
7214 เนื้อ
7215 รูป
7215 เก่ง
7215 เป็น
7215 เตะ
7215 ผม
7215 

7415 ศัพท์
7415 สิ่ง
7416 สูง
7416 พอ
7416 กับ
7416 มี
7416 เปรียบเทียบ
7416 คน
7416 จุดยืน
7417 เก่ง
7417 เรียน
7417 เล่น
7418 ให้
7418 ทำ
7419 เก่ง
7419 หน้า
7419 รอ
7419 ญาติ
7419 ตา
7419 ทำไม
7419 คน
7419 ห้อง
7420 กระจก
7420 ใช่
7420 สวย
7420 ชั้น
7421 เลือก
7421 หล่อ
7421 เป็น
7422 อย่าง
7422 กลับ
7422 มา
7422 เล่น
7423 อย่าง
7423 เลวร้าย
7423 รุนแรง
7423 รอ
7423 เป็น
7423 เสียใจ
7423 จุด
7423 ปลาย
7423 ทาง
7423 มี
7424 แรก
7424 พูด
7424 สูง
7424 ทำ
7424 ศัลยกรรม
7424 สิ่ง
7425 ขนม
7425 ดี
7425 ลูกค้า
7425 ดื่ม
7425 ไกล
7425 อาหาร
7425 อร่อย
7425 บริการ
7425 เครื่อง
7426 พูด
7426 ให้
7426 ทำ
7426 ยิ้ม
7426 คำ
7427 แบบ
7427 รัก
7427 วัน
7428 สุข
7428 มี
7428 เห็น
7428 ยิ้ม
7428 ใจ
7429 ให้
7429 ชอบ
7429 ทำ
7429 บอก
7429 ตอน
7429 กับ
7429 เชื่อ
7429 มี
7429 คิด
7429 คำ
7430 รุนแรง
7430 พิมพ์
7431 เก็บ
7431 ปัจจุบัน
7431 ดี
7431 แก้ไข
7431 มา
7431 กับ
7431 อดีต
7431 มี
7431 อนาคต
7432 ให้
7432 ดี
7432 ทำ
7432 พยายาม
7433 ออก
7434 รู้จัก
7434 ดี
7434 เชื่อ
7435 เรียน
7435 เดิน
7435 จ

7734 อย่าง
7734 เรียน
7734 อยู่
7734 ทำ
7734 มา
7734 ร้อย
7734 ห้อง
7735 ใช่
7736 พูด
7736 แบบ
7736 บาง
7736 สัมพันธ์
7736 อาจ
7736 คิด
7736 คำ
7737 ให้
7737 เขา
7737 เรื่อง
7737 ทำ
7737 บอก
7737 เสียใจ
7737 สัก
7737 คน
7738 เป็น
7738 สุดท้าย
7738 ตบ
7738 เห็น
7738 นางเอก
7738 ฉาก
7739 เข้าใจ
7739 ใส่
7739 รัก
7740 ทำ
7741 เสีย
7741 ทำร้าย
7741 บาง
7741 มี
7741 คน
7741 ผล
7742 อันดับ
7742 เลือก
7743 ตั้งใจ
7743 อยู่
7744 สวย
7747 ให้
7747 ทำ
7747 สัก
7747 หาก
7747 ทำไม
7747 มี
7747 คน
7747 คำ
7748 มี
7749 ตอบ
7750 ระยะ
7750 เป็น
7750 ทาง
7751 คิด
7752 พูด
7752 ยุ่ง
7752 เข้าใจ
7752 เสียใจ
7752 แบบ
7752 มด
7752 รัก
7752 ตอน
7752 มี
7752 แม่
7754 ให้
7754 แก้
7754 ตบ
7755 ปาก
7755 อย่าง
7755 บอก
7755 เป็น
7756 บีบ
7757 จริง
7757 ฝัน
7757 สู้
7757 ใช่
7758 ให้
7758 จำ
7758 คน
7759 หมู
7759 ให้
7759 ขอ
7759 จูบ
7759 จริง
7759 รอบ
7759 ก
7759 ลง
7759 เลว
7760 งาน
7760 มา
7760 เสร็จ
7761 สรุป
7761 มี
7761 งง
7762 ขอ
7763 หยุด
7763 แฟน
7763 สงสาร
7763 ผอม
7763 ลง
7764 รัก
7765 ดู
7765 วัน
776

In [246]:
emodict

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [0]])

In [247]:
emodict.shape

(8000, 1)

In [248]:
train_model(emodict,y,5)

Prediction
[1 1 1 ... 1 1 1]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.935 0.1281699794379712 0.22543701024713683

Prediction
[1 1 1 ... 1 1 1]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.93 0.12845303867403315 0.22572815533980584

Prediction
[1 1 1 ... 1 1 1]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.95 0.1296043656207367 0.22809123649459787

Prediction
[1 1 1 ... 1 1 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.92 0.125426039536469 0.220755848830234

Prediction
[1 1 1 ... 1 1 1]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.92 0.12449255751014884 0.21930870083432655


Average
0.931 0.1272291961558718 0.22386419034922023


In [249]:
def findDictFeatureNormalized(dictionary):
    N = len(wordDictList)
    count = 0.0
    dict_feature = [0 for i in range(N)]
    for idx, wordDict in enumerate(wordDictList):
        for word, val in wordDict.items():
            if val > 0:
                if word in dictionary:
                    dict_feature[idx] = dict_feature[idx] + 1
                    print(idx,word,val)
    for idx, doc in enumerate(tokenized_docs):
        n = len(doc)
        dict_feature[idx] = dict_feature[idx]/n
    return np.array(dict_feature).reshape(N,1)

In [250]:
emodictnormalized = findDictFeatureNormalized(emowords)

0 สอน 1
0 ดู 1
0 สัก 1
0 ก่อน 1
0 คน 1
1 รับ 1
2 ต่อ 1
2 ทำ 2
2 เป็น 2
2 แกล้ง 2
2 คน 1
3 อยู่ 1
3 บอก 2
3 พระ 1
3 ตอน 2
3 กรุ 1
4 แถว 1
4 แสดง 1
4 ซื้อ 1
4 มา 2
4 ข้าว 1
4 อาหาร 1
4 ใช่ 1
4 กิน 1
4 คน 1
4 ออก 1
4 ลง 1
4 ห้อง 1
5 หก 1
5 ด่า 1
6 กระโดด 1
6 อย่าง 1
6 หน้า 1
6 เป็น 1
6 กรรมการ 1
6 ตอแหล 1
7 หมู 1
7 แมว 3
7 หนวด 1
7 เบี้ยว 1
7 กัด 1
7 โทรศัพท์ 1
7 ทำไม 1
7 หมา 1
7 หลอด 1
8 ช่าง 1
9 พูด 1
9 กระทำ 1
9 กับ 1
9 แตกต่าง 1
9 คำ 1
10 แรก 1
10 ให้ 1
10 ปาก 1
10 ดัง 1
10 ทำ 1
10 บอก 1
10 แบบ 1
10 ตาย 1
10 แฟน 1
10 เรียก 3
10 ชื่อ 1
10 เพื่อน 2
10 ตีน 1
10 อารมณ์ 1
11 ฟัน 1
11 หา 1
12 เปิด 1
12 ไฟ 2
12 อยู่ 1
12 ตอน 1
12 มา 1
12 สาม 1
12 ดับ 1
12 โทรศัพท์ 1
12 อ่าน 2
13 ให้ 1
13 ใหม่ 2
13 เป็น 1
13 ตอน 1
13 มา 1
13 หลุด 1
13 ค้าง 1
14 คู่ 1
14 เหยียบ 1
14 ใส่ 1
14 ซื้อ 1
14 มา 1
15 ให้ 1
15 ชอบ 1
15 พ่อ 1
15 ดี 1
15 เป็น 1
15 ชื่อ 1
15 ตรง 1
15 คน 1
15 แม่ 1
16 ชนะ 1
16 เป่า 1
16 ทำไม 1
17 ให้ 1
17 ทำ 1
17 เป็น 1
17 ลูกศร 1
17 มี 1
17 ปะ 1
17 วาด 1
18 เป็น 1
19 หัว 1
19 ดี 1
19 ทำ 1

195 ออก 1
196 บ้าน 1
196 อยู่ 1
196 ทำ 1
196 กับ 1
196 นอน 1
196 ทำไม 1
196 น้ำหนัก 1
196 วัน 1
196 กิน 1
197 ดู 1
197 หัว 1
197 เต็ม 1
197 ขนาด 1
197 ผม 1
197 เรียก 1
197 ป้า 2
198 ชา 1
198 แห้ง 1
198 กิน 1
198 สอง 1
199 ชอบ 1
199 พอ 1
199 ดี 1
199 ทำ 2
199 เป็น 2
199 ตอน 1
199 กวน 1
199 เริ่ม 1
199 ทำไม 1
200 ทำ 2
200 เด็ก 1
200 แบบ 1
200 มา 1
200 เรียก 1
200 เรียบร้อย 1
201 รับ 1
201 นั่ง 1
201 เวลา 1
201 มา 1
201 วัน 1
202 ใช่ 1
203 เดิน 1
203 เขา 2
203 สัก 1
203 เวลา 1
203 น้ำ 1
203 กับ 1
203 จับ 1
203 เห็น 1
203 คน 1
204 เป็น 1
204 หิว 1
204 ไก่ 1
204 เริ่ม 1
204 ถุง 1
205 เดือน 1
205 ขโมย 1
205 ตอน 1
205 ใช้ 2
206 ต่อ 1
206 มา 1
206 ตาย 1
206 สะดุด 1
207 ชอบ 1
207 ฝน 1
207 ทำไม 1
207 ตก 1
207 วัน 1
208 ยัก 1
208 งาน 1
208 ตอน 1
208 มา 1
208 ตาย 1
208 ด่า 1
208 บ่น 1
210 มา 1
210 บ่น 2
211 รีบ 1
211 หนังสือ 1
211 ทำ 1
211 งาน 2
211 เรื่อย 1
211 เวลา 1
211 งอก 1
211 มา 1
211 มี 1
211 อ่าน 1
211 ชั้น 2
212 ขอ 1
212 จริง 1
212 แฟน 1
212 กับ 1
212 ทำไม 1
212 มี 1
212 พฤติกรรม 1
212 ค

375 ทีม 1
376 รถ 1
376 ให้ 1
376 บ้าน 2
376 อยู่ 1
376 สนาม 1
376 ปล่อย 1
376 ลม 1
376 นอน 1
376 ใช่ 1
376 วัน 1
376 แข่ง 1
376 ตาม 1
376 คน 1
376 แม่ 1
377 ดี 1
378 พูด 1
378 ต่อ 1
378 เรื่อง 1
378 มาก 1
378 หน้า 1
378 บอก 1
378 แบบ 1
378 มา 1
378 เลื่อน 1
378 อ่าน 1
378 ลง 1
379 อยู่ 1
379 ทำ 1
379 ดับ 1
379 รายงาน 1
380 คัน 1
381 มาก 1
381 กับ 1
381 กล้า 1
381 ก 1
381 เล่น 1
382 ซ่อม 1
382 เรือ 1
382 ยิ้ม 1
383 ใส่ 1
383 แบบ 1
383 ขนาด 1
383 กล้า 1
383 ผนัง 1
383 คน 1
384 ให้ 1
384 ทำ 2
384 บอก 1
384 งาน 1
384 มา 1
384 ง่าย 1
385 อ่าน 1
386 โกรธ 1
386 ออก 1
387 โคตร 1
387 กับ 1
388 แทง 1
388 เหลา 1
388 หน้า 1
388 ทำ 1
388 แหลม 1
388 เผา 1
388 มา 1
388 สงสาร 1
388 ก่อน 1
389 ชอบ 1
389 เกลียด 1
389 อยู่ 1
389 ทำ 2
389 จริง 1
389 มา 2
389 รัด 1
389 ขั้นตอน 2
389 ใหญ่ 2
389 มี 2
389 ตาม 1
389 ใช้ 1
390 แบบ 1
390 ทำไม 1
392 ให้ 1
393 ยาง 1
393 พูด 1
393 มา 1
393 เชียร์ 1
394 มา 1
394 ทำไม 2
394 เห็น 1
395 ให้ 1
395 ชีวิต 2
395 มา 1
395 กับ 1
395 ก่อน 1
395 คน 1
396 ให้ 1
396 ไฟ 1
396 มาก

553 ตอน 1
553 มา 1
553 สถานะ 1
553 ตี 1
553 ตรง 1
554 มาก 1
554 ดอก 1
554 กลับ 1
554 มา 1
554 กับ 1
555 รับผิดชอบ 1
555 อยู่ 1
555 ทำ 2
555 เป็น 1
555 ตอน 1
555 นิสัย 1
555 คน 2
555 คิด 1
556 หิว 1
557 ศิลปิน 1
557 เป็น 2
557 เสียใจ 3
557 รัก 1
557 แฟน 2
557 ด่า 3
557 เพื่อน 2
557 คน 1
558 ใกล้ 1
558 อยู่ 1
558 นั่ง 1
558 อิจฉา 1
558 แบบ 1
558 ลำบาก 1
558 เห็น 1
558 บาย 1
559 กับ 1
560 สอบ 1
560 เสร็จ 1
562 ลืม 1
562 เหลือ 1
562 เหยียบ 1
563 ให้ 1
563 ทำ 1
563 งาน 1
563 ราคา 1
563 ซื้อ 1
563 มา 1
563 เด้ง 1
563 เครื่อง 1
563 ออก 1
564 ดู 1
564 บอก 1
564 สนใจ 1
564 กับ 1
565 คน 1
566 ดู 2
566 มุม 1
566 เป็น 1
566 หนี 1
566 เอ็น 1
566 คอ 1
566 สภาพ 1
567 กับ 1
567 วัน 1
568 ยาก 1
569 ให้ 1
570 อย่าง 1
571 อย่าง 1
571 มาก 1
571 ขว้าง 1
571 หยิบ 1
571 ห้อง 2
572 ทำไม 1
573 ยาก 1
573 ดี 1
573 เป็น 2
573 คน 2
573 เลว 1
574 วัน 1
574 คน 1
575 แรก 1
575 กลม 1
575 แก้ม 1
575 หน้า 1
575 บอก 1
575 เป็น 1
575 สมัย 1
575 แบบ 1
575 รัก 1
575 หยิก 1
575 คน 1
576 เงียบ 1
576 สวย 1
577 ตั้งใจ 1
577 มา 

719 หี 1
719 โทรศัพท์ 1
719 หมา 1
720 หวัง 1
720 ให้ 1
720 สรุป 1
720 ทำ 1
720 ซับ 1
720 แบบ 1
720 วัน 1
720 คน 1
721 เรียน 1
721 เกลียด 1
721 จัด 1
721 บัญชี 1
721 วัน 1
723 ปัญญา 1
724 ข่าว 1
724 ดู 1
724 ระเบิด 1
724 แบบ 1
724 วาง 1
724 ร้าย 1
724 ใต้ 1
724 คน 1
725 คน 1
726 รับ 1
726 บน 1
726 หน้า 2
726 เด็ก 1
726 สัก 2
726 น้อย 1
726 วัน 3
726 คน 1
726 ตีน 2
727 รางวัล 1
728 พอ 1
728 เลิก 1
728 อ่าน 1
729 ใหม่ 1
729 หน้า 1
729 ทำ 1
729 ลูกค้า 1
729 งาน 1
729 ขโมย 1
729 ด้าน 1
729 มา 1
729 คน 2
730 โหล 1
730 ฟัง 1
730 ดี 1
730 ซื้อ 1
731 ให้ 1
731 เหลือ 1
731 ช่องว่าง 1
731 พัน 3
731 บัตร 2
731 แพง 1
731 ไล่ 1
731 เต็ม 1
731 ราคา 1
731 มา 1
731 ก่อน 1
731 รอบ 1
731 ตาม 1
731 ลง 1
732 อย่าง 1
732 ยุ่ง 1
732 มา 1
732 กับ 1
732 ชาย 1
733 รถ 1
733 เบรค 1
734 ไอ 1
734 ทิ้ง 1
734 เล่น 1
735 ให้ 1
735 พ่อ 1
735 มาก 1
735 ทำ 1
735 ด่า 1
736 บอก 1
736 เหว 1
736 มา 1
736 เห็น 1
737 รูป 1
737 นั่ง 1
737 คิ้ว 1
737 เสร็จ 1
737 ผูก 1
738 รอ 1
738 เป็น 1
738 สัก 1
738 ผม 1
738 ชม 1
739 ให้ 2
739

...

7074 รับ 1
7074 ขอ 1
7074 ทาง 1
7074 มี 1
7074 อ่าน 1
7074 เด็ดขาด 1
7076 ให้ 1
7076 คน 1
7077 ให้ 1
7077 ต่อ 1
7077 เขา 1
7077 เลือก 1
7077 พอ 1
7077 ดี 2
7077 หยุด 1
7077 เวลา 1
7077 มา 1
7077 แฟน 1
7077 ดิ้น 1
7077 ก่อน 1
7077 ดูแล 1
7077 มี 1
7078 ปิด 1
7078 มา 1
7078 น้ำหนัก 1
7079 หัว 1
7079 นก 1
7079 น้ำ 2
7079 ขนาด 1
7079 มี 2
7079 ทะเล 2
7079 โบราณ 1
7079 คน 1
7079 ตีน 1
7079 สี 2
7079 เผ่า 1
7080 รถ 1
7080 ให้ 2
7080 ต่อ 1
7080 นั่ง 1
7080 เป็น 2
7081 ให้ 2
7081 ปัญญา 1
7081 ทำ 1
7081 อิจฉา 2
7081 ดูแล 1
7081 มี 2
7081 คน 1
7081 สำคัญ 1
7082 รับ 1
7082 ทำ 1
7082 คน 1
7082 เลว 1
7083 ให้ 1
7083 สูง 1
7083 รู้จัก 1
7083 เด็ก 1
7083 โทร 1
7083 คอน 1
7083 มี 2
7083 คน 1
7084 ชอบ 1
7084 รัก 1
7084 แฟน 1
7084 หนู 2
7084 มี 1
7084 เห็น 1
7084 คน 1
7085 รัก 1
7085 เชื่อ 1
7085 ติ่ง 1
7086 ช้ำ 1
7086 เป็น 1
7086 เจ็บ 1
7087 มาก 1
7087 มา 1
7087 ตอบ 1
7087 หนู 1
7087 ก่อน 1
7087 สำคัญ 2
7087 คำ 1
7088 หน้า 1
7088 แรง 1
7088 ถูก 1
7088 ตบ 1
7088 ใช่ 1
7088 รู้สึก 1
7089 เดิน 1
7089 สัก 

7299 หญิง 2
7299 เพศ 1
7299 โง่ 2
7299 มี 1
7299 คน 1
7299 คำ 1
7300 ตั้งใจ 1
7300 เรื่อง 1
7300 หนังสือ 1
7300 จริง 1
7300 สอบ 1
7300 อ่าน 2
7302 ผม 1
7302 ใช่ 1
7303 แรง 1
7303 มือ 1
7303 จริง 1
7303 ตี 1
7304 ตอน 1
7304 สวย 1
7305 ถูก 1
7305 แบบ 1
7305 รัก 2
7305 พระ 1
7305 แฟน 1
7305 คน 2
7305 ง่าย 1
7306 พ่อ 1
7307 ใส่ 1
7307 กระดูก 1
7308 พูด 1
7308 เป็น 1
7308 ลม 1
7308 กลับ 1
7308 เชื่อ 1
7308 คำ 1
7309 หล่อ 1
7309 ผม 3
7309 เชื่อ 1
7309 เหล่ 1
7310 ใจเย็น 1
7310 อย่าง 1
7310 ชา 1
7310 วง 1
7310 ร่วม 1
7310 คู่หมั้น 1
7310 สู้ 1
7310 เพื่อน 1
7310 ตาม 1
7310 แน่นอน 1
7310 เล่น 1
7311 ให้ 1
7311 ต่อ 1
7311 อยู่ 1
7311 เป็น 1
7311 จริง 1
7311 จับ 1
7311 เพื่อน 1
7311 สถานการณ์ 1
7312 หญิง 1
7312 เป็น 1
7312 ชาย 1
7312 ตบ 2
7313 เสีย 1
7313 รัก 1
7313 ซื่อสัตย์ 1
7313 กับ 1
7314 เป็น 1
7314 กับ 1
7314 วัน 2
7314 ชง 1
7315 เรื่อง 2
7315 ดี 1
7315 บอก 1
7315 จริง 1
7315 ปิด 1
7315 ปกปิด 1
7315 ใช่ 1
7315 คน 1
7316 เลวร้าย 1
7316 รุนแรง 1
7316 เสียใจ 1
7317 ใกล้ 1
7317 หู 1
7317 ตาม 

7555 ออก 1
7556 เรื่อง 1
7556 หนังสือ 1
7556 อ่าน 1
7557 ดู 1
7557 รอ 1
7558 ดู 1
7558 ต่อ 1
7559 ให้ 2
7559 ชอบ 2
7559 สร้อย 1
7559 มาก 1
7559 เข้าใจ 1
7559 เพื่อน 2
7559 งง 2
7560 ภาค 1
7560 พูด 1
7560 อย่าง 1
7560 หล่อ 1
7560 ดี 1
7560 เป็น 1
7560 เรียบร้อย 1
7560 คน 1
7561 ใกล้ 1
7561 กลัว 1
7561 คน 1
7561 ศัตรู 1
7562 เรื่อง 1
7562 จริง 1
7562 เชื่อ 1
7562 ใช่ 1
7563 หล่อ 1
7564 ให้ 1
7564 เกลียด 1
7564 ใหม่ 1
7564 หน้า 1
7564 ทำ 1
7565 อย่าง 1
7565 อ่อน 1
7565 น้อย 1
7566 ใช่ 1
7568 ดู 1
7568 พันธมิตร 1
7568 เงียบ 1
7568 งาน 1
7568 เวลา 1
7568 มี 1
7568 หนัง 1
7568 ฉาก 1
7568 ทีม 1
7569 เลือก 1
7570 ยา 1
7570 ถูก 1
7570 แบบ 1
7570 มา 1
7570 จำ 1
7570 ท่อง 1
7570 ลง 1
7572 รัก 1
7572 กลับ 1
7572 โคตร 1
7573 เรื่อง 2
7573 หมาย 1
7573 ชั่ง 1
7573 ใช่ 1
7574 ให้ 1
7574 ซื้อ 1
7574 มา 1
7574 ข้าว 1
7574 แม่ 1
7575 หื่น 1
7575 ตลอด 1
7575 ง่าย 1
7576 กลัว 1
7576 เข้าใจ 1
7576 เชิญ 1
7577 คน 1
7578 เขา 2
7578 ทำ 1
7578 ยอม 1
7578 ตอบ 1
7578 ผม 3
7579 ขับ 2
7579 แรก 1
7579 รถ 1
7579 ให้ 

7809 ลืม 1
7809 ดี 3
7809 จริง 1
7809 จำ 1
7809 คิด 1
7810 รับ 1
7810 แรง 1
7810 ทำ 1
7810 คน 1
7811 มี 1
7812 รูป 1
7812 หญิง 1
7812 เสียง 1
7812 นั่ง 1
7812 ร้องไห้ 1
7812 เสียใจ 1
7812 แฟน 1
7812 ภาพ 1
7812 เคาะ 1
7812 ประตู 1
7812 มี 2
7812 เห็น 1
7812 สาว 1
7812 ห้อง 1
7813 เรียน 1
7813 เสื้อ 1
7813 ซื้อ 1
7813 ใหญ่ 1
7814 อย่าง 1
7814 ดวงจันทร์ 1
7814 อยู่ 1
7814 กระต่าย 1
7814 มี 1
7816 ชั่วโมง 1
7816 บอก 1
7816 กับ 1
7816 วัน 2
7816 มี 1
7817 มาก 1
7817 ดี 1
7818 เสีย 1
7818 รู้สึก 1
7818 คน 1
7819 ดู 2
7819 ต่อ 1
7819 พระเอก 1
7819 มือ 1
7819 สั่น 1
7819 ทน 1
7820 หมู 1
7820 หนวด 1
7820 เป็น 1
7821 ข้าว 1
7821 ขน 1
7822 ภาค 1
7822 ตาย 1
7822 คน 2
7823 ทำ 1
7824 เก่ง 1
7824 ภาษา 1
7824 คน 1
7825 กลับ 1
7825 มา 1
7826 รับ 1
7826 แรง 1
7826 ทำ 1
7826 มี 1
7826 ทน 2
7826 คน 2
7827 เหนื่อย 1
7827 แฟน 2
7827 มี 1
7828 ดู 1
7828 เพลง 1
7828 ทน 1
7829 โอกาส 1
7829 หมาย 1
7829 หน่อ 1
7829 โง่ 1
7829 มี 1
7829 ฉลาด 2
7830 เรียน 1
7830 เป็น 1
7830 รายการ 1
7831 ภาพ 1
7831 ใช่ 1
7831 คน 1

7994 เล่น 1
7995 เดียว 2
7995 รัก 1
7995 กับ 1
7995 ใจ 1
7996 ให้ 1
7996 เขียน 2
7996 ขนม 1
7996 บอก 1
7996 สวย 1
7997 ดู 1
7997 ให้ 1
7997 ทำ 1
7997 เวลา 1
7997 มา 1
7997 เผลอ 1
7997 ตา 1
7997 มี 1
7997 เห็น 1
7997 ตาม 1
7997 หา 1
7997 ใจ 1
7998 ท้อง 1
7998 ดาว 1


In [251]:
emodictnormalized

array([[0.23809524],
       [0.16666667],
       [0.20833333],
       ...,
       [0.30769231],
       [0.4       ],
       [0.        ]])

In [280]:
train_model(emodictnormalized,y,5)

Prediction
[1 1 0 ... 1 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.44 0.12054794520547946 0.189247311827957

Prediction
[0 0 0 ... 0 1 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.45 0.12032085561497326 0.189873417721519

Prediction
[1 0 0 ... 0 1 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.54 0.14173228346456693 0.22453222453222454

Prediction
[1 1 0 ... 1 1 1]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.54 0.14285714285714285 0.22594142259414227

Prediction
[0 1 1 ... 0 1 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.475 0.13194444444444445 0.20652173913043478


Average
0.48900000000000005 0.13148053431732137 0.20722322316125555


In [253]:
## concatenate emodict with tfidf

Xemo = np.concatenate((X,emodict), axis=1)
train_model(Xemo,y,5)

Prediction
[0 0 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.215 0.25748502994011974 0.23433242506811985

Prediction
[1 1 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.245 0.3223684210526316 0.27840909090909094

Prediction
[0 0 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.21 0.25 0.2282608695652174

Prediction
[0 0 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.255 0.32075471698113206 0.2841225626740947

Prediction
[1 1 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.2 0.272108843537415 0.23054755043227665


Average
0.225 0.28454340230225966 0.25113449972975993


In [254]:
## concatenate emodictnormalized with tfidf

XemoN = np.concatenate((X,emodictnormalized), axis=1)
train_model(XemoN,y,5)

Prediction
[0 0 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.215 0.25903614457831325 0.23497267759562843

Prediction
[1 1 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.245 0.3202614379084967 0.27762039660056653

Prediction
[0 0 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.21 0.25301204819277107 0.22950819672131145

Prediction
[0 0 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.25 0.32051282051282054 0.2808988764044944

Prediction
[1 1 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.195 0.2653061224489796 0.22478386167146974


Average
0.223 0.28362571472827625 0.24955680179869408


In [255]:
angrywords = np.array([df['Thai'],df['Anger']])
angrywords

array([['ผงะ', 'ลูกคิด', 'ละทิ้ง', ..., 'เกี่ยวกับสัตววิทยา',
        'สัตววิทยา', 'ซูม'],
       [0, 0, 0, ..., 0, 0, 0]], dtype=object)

In [256]:
angrywords.shape

(2, 14182)

In [257]:
angrywords[1][0]

0

In [258]:
angrywordslist = []
for i in range(len(angrywords[1])):
    if angrywords[1][i] == 1:
        angrywordslist.append(angrywords[0][i])
angrywordslist = np.array(angrywordslist).reshape(len(angrywordslist),1)

In [259]:
angrywordslist.shape

(1247, 1)

In [260]:
angry = findDictFeatureNormalized(angrywordslist)

4 แถว 1
6 ตอแหล 1
9 กับ 1
9 คำ 1
10 ดัง 1
12 ดับ 1
16 ชนะ 1
21 ร้อน 1
23 ตี 1
23 ทิ้ง 1
24 กับ 1
28 โกรธ 4
30 หยิ่ง 1
31 เงิน 1
35 ยิง 2
36 เงิน 1
38 ตบ 1
39 เกลียด 1
42 ตบ 1
43 ฟ้อง 1
47 เพ้อ 1
47 เลิก 1
49 ดับ 1
53 เกลียด 1
53 กับ 1
54 กับ 1
62 ทำร้าย 1
80 แห้ว 1
83 กับ 1
85 เงิน 2
94 กับ 1
96 จับ 1
99 แกะ 1
105 กับ 1
108 หยิ่ง 1
109 ตัด 1
110 กับ 1
115 ทิ้ง 1
116 ชน 1
118 ฆ่า 1
119 ร้าย 1
120 เกลียด 1
124 ระเบิด 1
124 บ่น 2
125 บ้า 1
131 กับ 1
133 กับ 1
135 เงิน 1
137 โรค 1
138 เสียใจ 1
138 กับ 1
138 จับ 1
146 ชิง 1
146 ชื่อเสียง 1
150 เลิก 1
152 เตะ 1
153 โกรธ 1
163 เลิก 1
165 คำ 2
167 วุ่นวาย 1
170 เลิก 2
170 กับ 1
173 เงิน 1
176 คำ 1
179 คำ 1
186 กับ 1
189 กับ 1
190 แทง 1
191 วุ่นวาย 2
196 กับ 1
203 กับ 1
203 จับ 1
205 ขโมย 1
206 สะดุด 1
208 บ่น 1
210 บ่น 2
212 กับ 1
218 ยุง 1
221 แม่บ้าน 1
224 เกลียด 1
224 ดูถูก 1
227 ตบ 1
228 บ้า 1
234 บ่น 1
236 พอดี 1
239 ทิ้ง 1
240 แถว 1
241 บ้า 1
241 แตก 1
242 ตัด 1
253 บ้า 1
257 ป่วย 1
257 โรค 1
257 กับ 1
258 กับ 1
259 แถว 1
263 แรง 1
264 ก

2436 หลอก 1
2437 คำ 1
2438 เงิน 1
2439 เกลียด 1
2440 เกลียด 1
2440 ประโยค 2
2440 คำ 1
2441 กับ 1
2442 กับ 1
2445 โกรธ 1
2451 กับ 1
2453 เกลียด 1
2455 กับ 1
2456 สั่น 1
2456 หลอก 1
2459 จับ 1
2466 เกลียด 2
2467 ร้อน 1
2468 กับ 1
2470 ผิดพลาด 1
2472 ตบ 1
2476 ไอ้ 1
2478 จับ 1
2480 กับ 1
2481 ร้าย 1
2481 คำ 1
2484 กับ 1
2484 คำ 1
2486 เจ็บ 1
2490 หยิ่ง 1
2492 ขวาง 1
2496 หลอกลวง 1
2496 กับ 1
2497 เกลียด 2
2498 ร้อน 1
2498 ถอน 1
2510 เจ็บ 1
2512 เจ็บ 1
2513 เกลียด 1
2516 เกลียด 1
2518 ร้าย 1
2519 เกลียด 1
2521 เกลียด 4
2522 ตบ 1
2529 คำ 1
2533 ทิ้ง 1
2539 กับ 1
2539 ทิ้ง 1
2542 ข้าม 1
2542 คำ 2
2543 กับ 1
2544 กับ 1
2548 เกลียด 1
2552 หลอก 1
2553 เกลียด 1
2560 เลิก 1
2563 เกลียด 1
2566 คำ 2
2568 บ้า 1
2569 ตี 1
2570 เกลียด 1
2570 ตอแหล 1
2573 เกลียด 2
2574 เงิน 1
2578 เกลียด 1
2580 กับ 1
2582 เจ็บ 1
2586 เลิก 1
2586 กับ 2
2588 ร้อน 1
2589 เลิก 1
2590 กับ 1
2591 กับ 1
2592 ทำร้าย 2
2593 จับ 1
2594 เกลียด 1
2595 คำ 1
2596 กลิ่น 1
2598 เงิน 2
2603 เกลียด 1
2603 เกี่ยวข้อง 1
2604 เกลียด 1
2607

4786 เงิน 1
4820 ถอย 1
4824 กลัว 1
4843 ร้อน 1
4843 บ่น 1
4850 ต่อสู้ 1
4850 กับ 1
4855 กับ 1
4855 พอดี 1
4857 อัด 1
4859 ทำร้าย 1
4861 แยก 1
4868 เงิน 1
4874 ทิ้ง 1
4874 รักษา 1
4877 ชน 1
4884 วุ่นวาย 1
4886 บ้า 1
4891 กับ 1
4892 เลิก 1
4893 บ้า 1
4897 เงิน 1
4900 ร้อน 1
4907 รั้ว 1
4909 ระเบิด 1
4915 ถอย 1
4919 กับ 1
4932 รักษา 1
4933 เงิน 1
4937 พอดี 1
4937 คำ 1
4939 ร้าย 1
4943 สละ 1
4969 พายุ 1
4970 เลิก 1
4991 กับ 1
4999 เงิน 1
5005 เอาแต่ใจ 1
5007 กับ 1
5008 กับ 1
5009 กับ 1
5011 หลอก 1
5019 คำ 2
5023 เจ็บ 1
5024 ธูป 1
5029 เงิน 1
5029 คำ 1
5038 เจ็บปวด 1
5046 กับ 1
5049 กับ 1
5053 ตัน 1
5057 กับ 2
5057 ทะเลาะ 1
5057 ทิ้ง 1
5058 กับ 1
5064 คำ 1
5069 เสียใจ 1
5070 คำ 2
5073 กับ 1
5078 กับ 1
5079 แถว 2
5079 กับ 2
5081 กับ 1
5083 หลอก 1
5085 กับ 2
5085 จับ 1
5091 กับ 1
5097 ทหาร 1
5101 กับ 1
5104 ชน 1
5108 กับ 1
5109 เลิก 1
5120 เงิน 1
5128 ดัง 1
5135 กับ 1
5136 เจ็บ 1
5137 คำ 1
5139 ตัด 1
5142 บุก 1
5144 เงิน 1
5148 ยุบ 1
5154 บาดแผล 1
5154 ฆ่า 1
5154 ทิ้ง 1
5155 แตก 1
5157 กับ 1


In [261]:
train_model(angry,y,5)

Prediction
[0 0 1 ... 1 0 1]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.32 0.16494845360824742 0.21768707482993196

Prediction
[0 0 0 ... 0 0 1]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.325 0.16971279373368145 0.2229845626072041

Prediction
[0 0 1 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.27 0.135678391959799 0.1806020066889632

Prediction
[0 0 1 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.23 0.1220159151193634 0.15944540727902945

Prediction
[1 0 0 ... 0 1 1]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.365 0.17298578199052134 0.23472668810289385


Average
0.302 0.15306826728232253 0.2030891479016045


In [262]:
Xangry = np.concatenate((X,angry),axis=1)
train_model(Xangry,y,5)

Prediction
[0 0 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.215 0.25443786982248523 0.23306233062330622

Prediction
[1 1 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.245 0.3223684210526316 0.27840909090909094

Prediction
[0 0 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.215 0.25 0.23118279569892475

Prediction
[0 0 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.25 0.31645569620253167 0.27932960893854747

Prediction
[1 1 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.2 0.2684563758389262 0.22922636103151864


Average
0.225 0.28234367258331494 0.2502420374402776


In [263]:
y2 = [ 0 for i in range(len(tokenized_docs))]
for i in range(1000,2000):
    y2[i]=1
y2 = np.array(y2)

In [264]:
train_model(X,y2,5)

Prediction
[0 0 0 ... 0 1 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.35 0.35175879396984927 0.3508771929824561

Prediction
[0 0 0 ... 0 0 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.43 0.3644067796610169 0.39449541284403666

Prediction
[0 0 0 ... 0 0 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.35 0.32710280373831774 0.3381642512077294

Prediction
[0 0 0 ... 0 0 1]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.405 0.3403361344537815 0.36986301369863017

Prediction
[0 0 0 ... 0 0 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.46 0.416289592760181 0.43705463182897863


Average
0.39899999999999997 0.3599788209166293 0.37809090051236627


In [265]:
y3 = [ 0 for i in range(len(tokenized_docs))]
for i in range(2000,3000):
    y3[i]=1
y3 = np.array(y3)

In [266]:
train_model(X,y3,5)

Prediction
[0 0 1 ... 0 0 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.34 0.3695652173913043 0.3541666666666667

Prediction
[1 0 1 ... 0 0 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.315 0.34615384615384615 0.3298429319371728

Prediction
[0 0 0 ... 0 0 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.395 0.3640552995391705 0.3788968824940048

Prediction
[0 0 1 ... 0 0 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.3 0.3015075376884422 0.30075187969924805

Prediction
[0 0 0 ... 0 0 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.395 0.3744075829383886 0.38442822384428227


Average
0.34900000000000003 0.35113789674223034 0.3496173169282749


In [267]:
y4 = [ 0 for i in range(len(tokenized_docs))]
for i in range(3000,4000):
    y4[i]=1
y4 = np.array(y4)
train_model(X,y4,5)

Prediction
[0 0 0 ... 0 0 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.33 0.3402061855670103 0.33502538071065996

Prediction
[0 0 0 ... 0 0 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.305 0.3465909090909091 0.324468085106383

Prediction
[0 0 0 ... 0 0 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.335 0.3489583333333333 0.34183673469387754

Prediction
[0 0 0 ... 1 0 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.35 0.33653846153846156 0.3431372549019608

Prediction
[0 0 0 ... 0 0 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.325 0.3385416666666667 0.33163265306122447


Average
0.32899999999999996 0.34216711123927623 0.33522002169482124


In [268]:
y5 = [ 0 for i in range(len(tokenized_docs))]
for i in range(4000,5000):
    y5[i]=1
y5 = np.array(y5)
train_model(X,y5,5)

Prediction
[0 0 0 ... 0 0 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.355 0.35148514851485146 0.35323383084577115

Prediction
[0 0 0 ... 0 0 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.37 0.36633663366336633 0.36815920398009955

Prediction
[0 0 0 ... 0 0 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.335 0.35638297872340424 0.34536082474226804

Prediction
[0 0 0 ... 0 0 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.355 0.36597938144329895 0.36040609137055835

Prediction
[0 0 0 ... 0 0 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.29 0.35365853658536583 0.3186813186813187


Average
0.341 0.3587685357860574 0.3491682539240032


In [269]:
y6 = [ 0 for i in range(len(tokenized_docs))]
for i in range(5000,6000):
    y6[i]=1
y6 = np.array(y6)
train_model(X,y6,5)

Prediction
[1 0 1 ... 0 0 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.285 0.23651452282157676 0.25850340136054417

Prediction
[0 0 0 ... 0 0 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.27 0.242152466367713 0.2553191489361702

Prediction
[0 0 1 ... 0 0 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.275 0.25 0.2619047619047619

Prediction
[0 0 0 ... 0 0 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.325 0.2653061224489796 0.29213483146067415

Prediction
[0 0 0 ... 0 1 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.315 0.2876712328767123 0.30071599045346065


Average
0.294 0.25632886890299633 0.2737156268231222


In [270]:
y7 = [ 0 for i in range(len(tokenized_docs))]
for i in range(6000,7000):
    y7[i]=1
y7 = np.array(y7)
train_model(X,y7,5)

Prediction
[1 0 0 ... 0 0 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.29 0.3515151515151515 0.31780821917808216

Prediction
[0 0 0 ... 0 1 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.245 0.30625 0.27222222222222225

Prediction
[0 0 0 ... 0 0 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.335 0.41358024691358025 0.3701657458563536

Prediction
[0 0 0 ... 0 0 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.305 0.41216216216216217 0.3505747126436782

Prediction
[0 0 0 ... 0 0 0]
y
[0 0 0 ... 0 0 0]
scores: recall, precision, f1
0.295 0.3881578947368421 0.3352272727272727


Average
0.29399999999999993 0.37433309106554724 0.3291996345255218


In [292]:
y8 = [ 0 for i in range(len(tokenized_docs))]
for i in range(7000,8000):
    y8[i]=1
y8 = np.array(y8)
train_model(X,y8,5)

Prediction
[0 0 0 ... 1 0 1]
y
[0 0 0 ... 1 1 1]
scores: recall, precision, f1
0.39 0.3786407766990291 0.3842364532019705

Prediction
[0 0 0 ... 1 0 0]
y
[0 0 0 ... 1 1 1]
scores: recall, precision, f1
0.39 0.39593908629441626 0.39294710327455923

Prediction
[0 0 1 ... 0 1 1]
y
[0 0 0 ... 1 1 1]
scores: recall, precision, f1
0.385 0.34841628959276016 0.3657957244655582

Prediction
[0 0 0 ... 1 1 1]
y
[0 0 0 ... 1 1 1]
scores: recall, precision, f1
0.435 0.43283582089552236 0.4339152119700748

Prediction
[0 0 0 ... 0 0 0]
y
[0 0 0 ... 1 1 1]
scores: recall, precision, f1
0.37 0.3645320197044335 0.36724565756823824


Average
0.394 0.38407279863723226 0.3888280300960802


In [291]:
train_model(X,y,5)

Prediction
[0 0 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.215 0.25748502994011974 0.23433242506811985

Prediction
[1 1 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.245 0.32450331125827814 0.2792022792022792

Prediction
[0 0 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.21 0.25 0.2282608695652174

Prediction
[0 0 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.255 0.32075471698113206 0.2841225626740947

Prediction
[1 1 0 ... 0 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.2 0.272108843537415 0.23054755043227665


Average
0.225 0.284970380343389 0.2512931373883976


In [273]:
from sklearn.naive_bayes import GaussianNB

def train_model_nb(X,y,n):
    features = X
    label =y
    
    R = 0.0
    P = 0.0
    F = 0.0
    
    skf = StratifiedKFold(n_splits=n, random_state=30, shuffle=True)
    
    for train_index, test_index in skf.split(features, label):
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = label[train_index], label[test_index]
        tclf = GaussianNB().fit(X_train,y_train)
        predict = tclf.predict(X_test)
        
        r=recall_score(y_test,predict)
        p=precision_score(y_test,predict)
        f=f1_score(y_test,predict)
        
        R = R + r
        P = P + p
        F = F + f
        
        print('Prediction')
        print(predict)
        print('y')
        print(y_test)
        print('scores: recall, precision, f1')
        print(r,p,f)
        print()
        
    print()
    print('Average')
    print(R/n,P/n,F/n)

In [293]:
train_model_nb(X,y,5)

Prediction
[1 0 0 ... 0 0 1]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.53 0.14950634696755993 0.2332233223322332

Prediction
[1 1 0 ... 1 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.515 0.14148351648351648 0.22198275862068964

Prediction
[0 1 0 ... 1 0 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.48 0.13370473537604458 0.20915032679738566

Prediction
[0 0 1 ... 1 0 1]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.53 0.14095744680851063 0.22268907563025211

Prediction
[0 1 1 ... 1 1 0]
y
[1 1 1 ... 0 0 0]
scores: recall, precision, f1
0.565 0.15374149659863945 0.24171122994652405


Average
0.5239999999999999 0.14387870844685421 0.2257513426654169


In [299]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)

In [307]:
p = clf.predict(X_test)

In [308]:
len(p)

3200

In [304]:
for idx,y in enumerate(y_test):
    if y==1: print(idx)

3
5
20
22
23
27
35
44
46
54
58
66
70
73
86
88
98
104
109
123
136
140
145
146
150
154
160
180
183
199
200
201
205
207
219
228
229
234
235
236
238
241
249
260
261
276
283
285
295
304
312
314
331
340
346
347
349
363
379
395
401
409
411
415
416
433
440
443
451
457
458
461
463
464
471
472
478
480
486
494
514
515
516
518
540
543
565
570
575
580
594
604
607
609
613
616
617
622
629
636
656
658
660
670
675
687
704
709
710
715
718
733
735
737
750
761
762
764
771
782
791
797
798
810
811
826
835
848
854
857
865
892
918
926
934
936
942
961
967
976
993
999
1000
1006
1021
1024
1026
1029
1030
1032
1040
1044
1053
1070
1073
1080
1081
1105
1111
1113
1118
1119
1127
1137
1148
1152
1155
1157
1160
1166
1181
1185
1196
1199
1210
1219
1234
1242
1256
1279
1292
1310
1314
1324
1325
1335
1344
1356
1359
1368
1387
1391
1394
1419
1424
1452
1455
1475
1487
1493
1553
1557
1562
1570
1574
1591
1609
1610
1611
1612
1632
1639
1652
1653
1654
1659
1669
1685
1702
1726
1729
1733
1735
1738
1751
1762
1766
1769
1787
1788
1794
1802
1

In [309]:
print(recall_score(y_test,p),precision_score(y_test,p),f1_score(y_test,p))

0.046228710462287104 0.4318181818181818 0.08351648351648351


In [310]:
for idx,y in enumerate(p):
    if y==1: print(idx)

239
317
428
429
456
457
514
616
771
808
934
1006
1210
1284
1561
1562
1611
1626
1716
1802
1804
1840
1853
1874
1934
1978
2076
2105
2275
2302
2326
2397
2415
2432
2568
2569
2576
2691
2725
2885
2974
3038
3046
3120
